#Modules

In [1]:
from pandas.core.common import random_state
import pandas as pd
import numpy as np
import gensim
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
%matplotlib inline
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency
import seaborn as sns
import spacy  # For preprocessing
import warnings
warnings.filterwarnings('ignore')
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


random_state=2022

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
bow_data=pd.read_csv('/content/drive/MyDrive/NLP/Dataset/bow_dataset.csv')
tf_idf_data=pd.read_csv('/content/drive/MyDrive/NLP/Dataset/tfidf_dataset.csv')
w2v_df=pd.read_csv('/content/drive/MyDrive/NLP/Dataset/w2v_df.csv')
d2v_df=pd.read_csv('/content/drive/MyDrive/NLP/Dataset/d2v_df.csv')

In [4]:
bow_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,1010,1011,1012,1013,1014,1015,1017,1019,1022,1023
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.001996,0.001209,1.0,0.166667,0.000000,0.192308,0.005571
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.001996,0.001209,1.0,0.187500,0.000000,0.038462,0.008959
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.019871,0.014965,0.5,0.520833,0.000000,0.000000,0.030561
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.019871,0.014965,1.0,0.583333,0.000000,0.000000,0.029139
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.019871,0.014965,1.0,0.583333,0.057143,0.000000,0.029139


In [4]:
dropped = ['Unnamed: 0']
data = bow_data.drop(dropped,axis=1)
bow_data=data

#bow

In [28]:
# y = np.ravel(data[806])
y=bow_data.iloc[ :, -1:]  #Target Variable          
X=bow_data.iloc[: , :-1]  #Feature Matrix 

# y.head()

In [12]:
#converted the dataframe

X=pd.DataFrame(X)
y=pd.DataFrame(y)

In [13]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2022)

In [14]:
def hyperparameter_Tuning():
      model_rand = RandomForestRegressor()
      n_estimators = [5,20,50,100] # number of trees in the random forest
      max_features = ['auto', 'sqrt'] # number of features in consideration at every split
      max_depth = [int(x) for x in np.linspace(10, 120, num = 12)] # maximum number of levels allowed in each decision tree
      min_samples_split = [2, 6, 10] # minimum sample number to split a node
      min_samples_leaf = [1, 3, 4] # minimum sample number that can be stored in a leaf node
      bootstrap = [True, False] # method used to sample data points
      random_grid = {'n_estimators': n_estimators,
      'max_features': max_features,
      'max_depth': max_depth,
      'min_samples_split': min_samples_split,
      'min_samples_leaf': min_samples_leaf,
      'bootstrap': bootstrap}
      rf_random = RandomizedSearchCV(estimator = model_rand,param_distributions = random_grid,n_iter = 100, cv = 5, verbose=2, random_state=35, n_jobs = -1)
      rf_random.fit(x_train, y_train)
      return rf_random.best_params_

In [15]:
rnd_clf = RandomForestRegressor(n_estimators=20, min_samples_split=10, min_samples_leaf=4,max_features='sqrt',max_depth=10,bootstrap=True)
rnd_clf.fit(x_train, y_train)
y_pred_rnd_clf = rnd_clf.predict(x_test)

In [16]:
def plot_feature_importance(importance,names,model_type):

#Create arrays from feature importance and feature names
  feature_importance = np.array(importance)
  feature_names = np.array(names)

  #Create a DataFrame using a Dictionary
  data={'feature_names':feature_names,'feature_importance':feature_importance}
  fi_df = pd.DataFrame(data)

  #Sort the DataFrame in order decreasing feature importance
  fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

  #Define size of bar plot
  plt.figure(figsize=(20,20))
  #Plot Searborn bar chart
  sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])
  #Add chart labels
  plt.title(model_type + 'FEATURE IMPORTANCE')
  plt.xlabel('FEATURE IMPORTANCE')
  plt.ylabel('FEATURE NAMES')

# plot_feature_importance(rnd_clf.feature_importances_,x_train.columns,'RANDOM FOREST')

In [17]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred_rnd_clf))
print("RMSE: %f" % (rmse))

R2=r2_score(y_test,y_pred_rnd_clf)
print("R2 Score: %f" % (R2))
print()

RMSE: 0.045439
R2 Score: 0.539122



#tfid

In [18]:
tf_idf_data.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,1010,1011,1012,1013,1014,1015,1017,1019,1022,1023
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.001996,0.001209,1.0,0.166667,0.000000,0.192308,0.005571
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.001996,0.001209,1.0,0.187500,0.000000,0.038462,0.008959
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.019871,0.014965,0.5,0.520833,0.000000,0.000000,0.030561
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.019871,0.014965,1.0,0.583333,0.000000,0.000000,0.029139
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.019871,0.014965,1.0,0.583333,0.057143,0.000000,0.029139


In [ ]:
dropped = ['Unnamed: 0']
data = tf_idf_data.drop(dropped,axis=1)
tf_idf_data=data

In [19]:
y=tf_idf_data.iloc[ :, -1:]  #Target Variable          
X=tf_idf_data.iloc[: , :-1]  #Feature Matrix 

# y.head()

In [20]:
#converted the dataframe

X=pd.DataFrame(X)
y=pd.DataFrame(y)

In [23]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2022)

In [22]:
rnd_clf = RandomForestRegressor(n_estimators=20, min_samples_split=10, min_samples_leaf=4,max_features='sqrt',max_depth=10,bootstrap=True)
rnd_clf.fit(x_train, y_train)
y_pred_rnd_clf = rnd_clf.predict(x_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred_rnd_clf))
print("RMSE: %f" % (rmse))

R2=r2_score(y_test,y_pred_rnd_clf)
print("R2 Score: %f" % (R2))
print()

RMSE: 0.045888
R2 Score: 0.529953



#w2v

In [24]:
w2v_df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,1011,1012,1013,1014,1015,1016,1018,1020,1023,1024
0,0,0.440175,0.485191,0.587695,0.585352,0.398419,0.435185,0.437130,0.410533,0.516202,...,0.0,0.0,0.0,0.001996,0.001209,1.0,0.166667,0.000000,0.192308,0.005571
1,1,0.439598,0.391996,0.498240,0.555270,0.422490,0.516015,0.405562,0.436408,0.577282,...,0.0,0.0,0.0,0.001996,0.001209,1.0,0.187500,0.000000,0.038462,0.008959
2,2,0.498305,0.628040,0.523453,0.595455,0.576013,0.499553,0.303011,0.306349,0.461778,...,0.0,1.0,0.0,0.019871,0.014965,0.5,0.520833,0.000000,0.000000,0.030561
3,3,0.677892,0.666789,0.506650,0.476117,0.498457,0.381619,0.401607,0.412709,0.543147,...,0.0,0.0,0.0,0.019871,0.014965,1.0,0.583333,0.000000,0.000000,0.029139
4,4,0.571104,0.508060,0.440763,0.579581,0.470572,0.444646,0.463706,0.399712,0.534334,...,0.0,0.0,0.0,0.019871,0.014965,1.0,0.583333,0.057143,0.000000,0.029139


In [ ]:
dropped = ['Unnamed: 0']
data = w2v_df.drop(dropped,axis=1)
w2v_df=data

In [26]:
y=w2v_df.iloc[ :, -1:]  #Target Variable          
X=w2v_df.iloc[: , :-1]  #Feature Matrix 



X=pd.DataFrame(X)
y=pd.DataFrame(y)
for i in X.columns:
  X[i]=pd.to_numeric(X[i])
for i in y.columns:
  y[i]=pd.to_numeric(y[i])


x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=2022)

In [29]:
rnd_clf = RandomForestRegressor(n_estimators=100, min_samples_split=10 , min_samples_leaf=4, max_features='auto',max_depth=50,bootstrap=True)
rnd_clf.fit(x_train, y_train)
y_pred_rnd_clf = rnd_clf.predict(x_test)

rmse_rnd = np.sqrt(mean_squared_error(y_test, y_pred_rnd_clf))
print("RMSE: %f" % (rmse_rnd))

R2_rnd=r2_score(y_test,y_pred_rnd_clf)
print("R2 Score: %f" % (R2_rnd))

RMSE: 0.018853
R2 Score: 0.920658


#d2v

In [30]:
d2v_df.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,1211,1212,1213,1214,1215,1216,1218,1220,1223,1224
0,0,0.368360,0.557300,0.592499,0.579223,0.374454,0.460472,0.436838,0.615206,0.362663,...,0.0,0.0,0.0,0.001996,0.001209,1.0,0.166667,0.000000,0.192308,0.005571
1,1,0.409319,0.541178,0.523558,0.508291,0.320311,0.512994,0.401727,0.554929,0.387721,...,0.0,0.0,0.0,0.001996,0.001209,1.0,0.187500,0.000000,0.038462,0.008959
2,2,0.485085,0.542153,0.539946,0.550250,0.565587,0.401578,0.624751,0.308196,0.622911,...,0.0,1.0,0.0,0.019871,0.014965,0.5,0.520833,0.000000,0.000000,0.030561
3,3,0.644376,0.541030,0.576122,0.577396,0.501819,0.560930,0.561669,0.442864,0.521159,...,0.0,0.0,0.0,0.019871,0.014965,1.0,0.583333,0.000000,0.000000,0.029139
4,4,0.568752,0.729264,0.574859,0.483465,0.502268,0.463491,0.466647,0.407807,0.578999,...,0.0,0.0,0.0,0.019871,0.014965,1.0,0.583333,0.057143,0.000000,0.029139


In [31]:
dropped = ['Unnamed: 0']
data = d2v_df.drop(dropped,axis=1)
d2v_df=data

In [32]:
y=d2v_df.iloc[ :, -1:]  #Target Variable          
X=d2v_df.iloc[: , :-1]  #Feature Matrix 

X=pd.DataFrame(X)
y=pd.DataFrame(y)

for i in X.columns:
  X[i]=pd.to_numeric(X[i])
for i in y.columns:
  y[i]=pd.to_numeric(y[i])


x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [33]:
rnd_clf = RandomForestRegressor(n_estimators=100, min_samples_split=10 , min_samples_leaf=4, max_features='auto',max_depth=50,bootstrap=True)
rnd_clf.fit(x_train, y_train)
y_pred_rnd_clf = rnd_clf.predict(x_test)

rmse_rnd = np.sqrt(mean_squared_error(y_test, y_pred_rnd_clf))
print("RMSE: %f" % (rmse_rnd))

R2_rnd=r2_score(y_test,y_pred_rnd_clf)
print("R2 Score: %f" % (R2_rnd))

RMSE: 0.022301
R2 Score: 0.901318
